In [ ]:
from autumn.tools.project import get_project
from matplotlib import pyplot
from autumn.tools.plots.utils import REF_DATE
from autumn.models.covid_19.constants import BASE_DATETIME
from autumn.tools.calibration.targets import get_target_series
import pandas as pd
from autumn.tools.utils.pretty import pretty_print
from autumn.tools.inputs.covid_au.queries import get_dhhs_vaccination_numbers
from summer.utils import ref_times_to_dti

In [ ]:
region = "north-metro"
project = get_project("covid_19", region)

In [ ]:
custom_params = project.param_set.baseline#.update(dict(contact_rate = 0.1))
model = project.run_baseline_model(custom_params)
derived_df = model.get_derived_outputs_df()

In [ ]:
output = "notifications"
notif_dates, notif_values = get_target_series(project.calibration.targets, REF_DATE, output)

In [ ]:
# pretty_print(custom_params)

In [ ]:
fig = pyplot.figure(figsize=(12, 8))
axis = fig.add_subplot()
axis = derived_df[output].plot()
axis.scatter(notif_dates, notif_values, c="k")
fig.show()

In [ ]:
fig = pyplot.figure(figsize=(12, 8))
axis = fig.add_subplot()
axis.fill_between(derived_df.index, [0.] * len(derived_df), derived_df["proportion_vaccinated"], label="fully vaccinated")
axis.fill_between(derived_df.index, derived_df["proportion_vaccinated"], derived_df["at_least_one_dose_prop"], label="one dose only")
axis.fill_between(derived_df.index, derived_df["at_least_one_dose_prop"], [1.] * len(derived_df), label="unvaccinated")
vacc_times, vacc_coverage = get_dhhs_vaccination_numbers(region.upper().replace("-", "_"), start_age=0)
vacc_dates = ref_times_to_dti(BASE_DATETIME, [int(i) for i in vacc_times])  # Converting numpy ints returned
lagged_vacc_dates = ref_times_to_dti(BASE_DATETIME, [int(i) + 14 for i in vacc_times])
axis.plot(vacc_dates, vacc_coverage, color="k", label="actual vaccination")
axis.plot(lagged_vacc_dates, vacc_coverage, color="k", linestyle="--", label="lagged for immunity")
axis.tick_params(axis="x", labelrotation=45)
axis.legend()
